In [1]:
import numpy as np  # Module that simplifies computations on matrices
import matplotlib.pyplot as plt  # Module used for plotting
from pylsl import StreamInlet, resolve_byprop  # Module to receive EEG data



import bci_workshop_tools as BCIw  # Our own functions for the workshop
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second

In [30]:
def gen_normal_dataset(feature_matrix_0, feature_matrix_1):
    """Generate an EEG dataset for binary classification. 

    Allows external classification algorithm to be used

    Args:
        feature_matrix_0 (numpy.ndarray): array of shape (n_samples,
            n_features) with examples for Class 0
        feature_matrix_0 (numpy.ndarray): array of shape (n_samples,
            n_features) with examples for Class 1
    Returns:
        X: input EEG power data 
        Y: labeled samples

    """
    # Create vector Y (class labels)
    class0 = np.zeros((feature_matrix_0.shape[0], 1))
    class1 = np.ones((feature_matrix_1.shape[0], 1))

    # Concatenate feature matrices and their respective labels
    y = np.concatenate((class0, class1), axis=0)
    features_all = np.concatenate((feature_matrix_0, feature_matrix_1),
                                  axis=0)

    # Normalize features columnwise
    mu_ft = np.mean(features_all, axis=0)
    std_ft = np.std(features_all, axis=0)

    X = (features_all - mu_ft) / std_ft
    return X, y

In [31]:
def stream_muse_script(stimuli_time=30): #SPLIT INTO CONNECT STREAM AND RECORD AND PROCESS

    """ 1. CONNECT TO EEG STREAM """

    # Search for active LSL stream
    print('Looking for an EEG stream...')
    streams = resolve_byprop('type', 'EEG', timeout=2)
    if len(streams) == 0:
        raise RuntimeError('Can\'t find EEG stream.')

    # Set active EEG stream to inlet and apply time correction
    print("Start acquiring data")
    inlet = StreamInlet(streams[0], max_chunklen=12)
    eeg_time_correction = inlet.time_correction()

    # Get the stream info, description, sampling frequency, number of channels
    info = inlet.info()
    description = info.desc()
    fs = int(info.nominal_srate())
    n_channels = info.channel_count()

    # Get names of all channels
    ch = description.child('channels').first_child()
    ch_names = [ch.child_value('label')]
    for i in range(1, n_channels):
        ch = ch.next_sibling()
        ch_names.append(ch.child_value('label'))

    """ 2. SET EXPERIMENTAL PARAMETERS """

    # Length of the EEG data buffer (in seconds)
    # This buffer will hold last n seconds of data and be used for calculations
    buffer_length = 15

    # Length of the epochs used to compute the FFT (in seconds)
    epoch_length = 1

    # Amount of overlap between two consecutive epochs (in seconds)
    overlap_length = 0.8

    # Amount to 'shift' the start of each next consecutive epoch
    shift_length = epoch_length - overlap_length

    # Index of the channel (electrode) to be used
    # 0 = left ear, 1 = left forehead, 2 = right forehead, 3 = right ear
    index_channel = [0, 1, 2, 3]
    # Name of our channel for plotting purposes
    ch_names = [ch_names[i] for i in index_channel]
    n_channels = len(index_channel)

    # Get names of features
    # ex. ['delta - CH1', 'pwr-theta - CH1', 'pwr-alpha - CH1',...]
    feature_names = BCIw.get_feature_names(ch_names)

    # Number of seconds to collect training data for (one class)
    training_length = stimuli_time

    """ 3. RECORD TRAINING DATA """

    # Record data for mental activity 0
    #winsound.Beep(400,1000)
    eeg_data0, timestamps0 = inlet.pull_chunk(
            timeout=training_length+1, max_samples=fs * training_length)
    eeg_data0 = np.array(eeg_data0)[:, index_channel]

    print('Stimuli')

    # Record data for mental activity 1
    winsound.Beep(400,1000) # Beep sound
    eeg_data1, timestamps1 = inlet.pull_chunk(
            timeout=training_length+1, max_samples=fs * training_length)
    eeg_data1 = np.array(eeg_data1)[:, index_channel]

    # Divide data into epochs
    eeg_epochs0 = BCIw.epoch(eeg_data0, epoch_length * fs,
                             overlap_length * fs)
    eeg_epochs1 = BCIw.epoch(eeg_data1, epoch_length * fs,
                             overlap_length * fs)

    """ 4. COMPUTE FEATURES AND TRAIN CLASSIFIER """

    feat_matrix0 = BCIw.compute_feature_matrix(eeg_epochs0, fs)
    feat_matrix1 = BCIw.compute_feature_matrix(eeg_epochs1, fs)

    [X_eeg, y_eeg] = gen_normal_dataset(
            feat_matrix0, feat_matrix1)
    return X_eeg, y_eeg, ch_names

    winsound.Beep(500,1000)

In [36]:
[X_eeg, y_eeg, ch_names]=stream_muse_script(30)
winsound.Beep(400,1000)

Looking for an EEG stream...
Start acquiring data
Stimuli


In [33]:
type(ch_names[2])

str

In [37]:
feature_names=BCIw.get_feature_names(ch_names)
feature_names.append('Label')

In [38]:
import pickle as pickle
import pandas as pd
import os
to_Pandas=np.concatenate((X_eeg,y_eeg), axis=1)
EEG_data=pd.DataFrame(to_Pandas)
EEG_data.columns=[feature_names]

os.makedirs('tmp', exist_ok=True)

EEG_data.to_pickle('tmp/EEG_data.pkl')
EEG_data=pd.read_pickle('tmp/EEG_data.pkl')
EEG_data.head()

,delta-TP9,delta-AF7,delta-AF8,delta-TP10,theta-TP9,theta-AF7,theta-AF8,theta-TP10,alpha-TP9,alpha-AF7,alpha-AF8,alpha-TP10,beta-TP9,beta-AF7,beta-AF8,beta-TP10,Label
0,0.213514,2.399008,2.407957,0.393249,0.534341,0.836858,1.424246,0.657038,-0.467993,0.814466,0.643818,0.098162,-1.078152,0.973413,-0.246633,-0.845306,0.0
1,0.274502,0.921714,1.514658,0.496492,0.181557,-0.012250,0.626624,0.175984,-0.906638,0.832131,-0.332070,0.329700,-1.247669,-0.714387,-1.326559,-1.020843,0.0
2,0.069119,0.729497,1.285416,0.097301,-0.467740,0.760934,-0.579900,-0.558739,-1.301238,-0.218394,-0.055393,0.249293,-1.385920,0.074558,-1.289886,-0.749937,0.0
3,0.003141,0.480663,1.615700,-0.158431,-1.007132,1.443319,-0.212008,-0.532355,-1.779209,-1.227202,0.353502,-0.087381,-0.508575,-0.075375,-0.891891,-0.433480,0.0
4,-0.210349,0.119917,1.832329,-0.231751,-0.616393,1.045143,-0.127250,-0.344241,-1.137883,-1.686370,0.923469,-0.147632,0.351424,-1.111880,-1.395548,-0.580706,0.0
